In [2]:
class HoLee():
    # HoLee Model for pring fixed income products
    # P:=[P_1,P_2,...P_T] 
    # P_i is the the price of zero Coupon Bonds matured in i periods
    # Notional Amount is 1
    # sigma is the annualized std of short rates in decimals (!!not percentage).
    # delta is the time step for each period, e.g. 0.25 year
    #########################################################################
    #
    #   Node 0    Node 1      Node 2      Node 3       Node 4       Node 5
    #
    #                                                            0.066643
    #                                               0.0632758
    #                                  0.0616506                 0.0616437
    #                       0.056084                0.0582758
    #            0.05263               0.0566506                 0.0566437
    #0.04969                0.051084                0.0532758
    #            0.04763               0.0516506                 0.0516437
    #                       0.046084                0.0482758
    #                                  0.0466506                 0.0466437
    #                                               0.0432758
    #                                                            0.04164375
    #
    ##########################################################################
    def __init__(self):
        import numpy as np
        import pandas as pd
        
        self.P_zcb=np.nan
        self.sigma=np.nan
        self.delta=np.nan
        # The risk-neutral Prices Tree
        self.prices_tree=np.nan
        # The risk-neutral Interest Rates Tree
        self.rates_tree=np.nan
        self.thetas=np.nan
        self.compounding=np.nan

    def fit(self,P_zcb,sigma,delta,compounding=0):
        # if compounding=0 ,Continuously Compounding
        # if compounding=1, compounding 1/delta times a year.
        from scipy.optimize import fsolve
        import numpy as np
        import pandas as pd
        thetas=[]
        P=list(P_zcb)
        if compounding ==0:
            r0=np.log(P[0])/(-delta)
        else:
            r0=(1/P[0]-1)/delta
        for i,price in enumerate(P[1:]):
            p0=price
            func=(lambda t: self.myholee(r0,sigma,delta,thetas+[t],compounding)[0]-p0)
            new_theta=fsolve(func,0.02)
            thetas.append(new_theta[0])

        self.P_zcb=P_zcb
        self.sigma=sigma
        self.delta=delta
        self.thetas=thetas
        self.compounding=compounding

        self.rates_tree=self.myholee(r0,sigma,delta,thetas,compounding)[2]
        self.prices_tree=self.myholee(r0,sigma,delta,thetas,compounding)[1]


        return 
    
    def summary(self):
        print("Fitted Interest Rates Tree:")
        print(self.rates_tree)
        print("============================")
        print("Fitted Prices Tree:")
        print(self.prices_tree)

    
    def pricing(self,CFs,type='conditional',lag=1):
        import numpy as np
        import pandas as pd
        def discount(rr,TT):
            if self.compounding==0:
                return np.exp(-rr*TT)
            else:
                return 1/(1+rr*self.delta)**(TT/self.delta)
        # type: fixed, CFs are fixed, and given as a array [CFS_1,CF_2,...,CF_T]
        # type: conditional, CFs are contingent on j,r, and given as a function CFs(,r)
        # lag mean, for the functin type CFs, it CF_T = CFs(r_{T-lag})
        # lag=0, the contingent CF is paid instantly after the amount is decided
        # lag=1, means the contingent CF is paid 1 peirod after the amount is decided.
        if type=="fixed":
            assert len(CFs)==len(self.P_zcb), "Length of CFs are not equal to Length of Given Zero Coupon Bonds"
            prices=np.zeros(self.prices_tree.shape)
            layers=prices.shape[1]
            for j in np.arange(layers-2,-1,-1):
                for i in np.arange(j+1):
                    r=self.rates_tree.iloc[i,j]
                    prices[i,j]=discount(r,self.delta)*0.5*(prices[i,j+1]+prices[i+1,j+1])+discount(r,self.delta)*CFs[j]
        
        else:
            from inspect import isfunction
            assert isfunction(CFs), "For Non-Fixed payoffs, CFs must be a function!"

            
            prices=np.zeros(self.prices_tree.shape)
            layers=prices.shape[1]
            for j in np.arange(layers-2,-1,-1):
                for i in np.arange(j+1):
                    r=self.rates_tree.iloc[i,j]
                    # Pay instantly
                    if lag==0:
                        prices[i,j]=discount(r,self.delta)*0.5*(prices[i,j+1]+prices[i+1,j+1])+CFs(j,r)
                    # Pay 1 period after the r is realized
                    elif lag==1:
                        prices[i,j]=discount(r,self.delta)*0.5*(prices[i,j+1]+CFs(j,r)+prices[i+1,j+1]+CFs(j,r))
                    else:
                        print("lag must be 0 or 1!")
                        raise Error




        return [prices[0,0],pd.DataFrame(prices)]



    @staticmethod
    def myholee(r0,sigma,delta,thetas,compounding=0):
        import numpy as np
        import pandas as pd
        # r0 is the inital short rate
        # thetas are theta_0 to theta_T
        # delta is the time step
        # m is theta_(T+1)
        # compounding: 0: continuously compounding
        # compounding: 1: 1/delta times a year
        # return P[0,0],Prices, Risk_Neutral_Prices
        layers=len(thetas)+1
        Prices=np.zeros((layers+1,layers+1))
        Prices[:,-1]=np.ones(layers+1)
        InterestRates=np.zeros((layers,layers))

        def discount(rr,TT):
            if compounding==0:
                return np.exp(-rr*TT)
            else:
                return 1/(1+rr*delta)**(TT/delta)
        
        # thetas=thetas+[m]
        for j in np.arange(layers-1,-1,-1):
            for i in np.arange(j+1):
                kk=(j-2*i)*sigma*np.sqrt(delta)
                r=r0+np.sum([theta*delta for theta in thetas[:j]])+kk
                InterestRates[i,j]=r
                Prices[i,j]=0.5*(discount(r,delta)*Prices[i,j+1]+discount(r,delta)*Prices[i+1,j+1])
        import pandas as pd
        return Prices[0,0],pd.DataFrame(Prices),pd.DataFrame(InterestRates)
        
        






In [3]:
hl1=HoLee()

In [4]:
hl1.summary()

Fitted Interest Rates Tree:
nan
Fitted Prices Tree:
nan


In [5]:
a1=  1/(1+0.04969*0.25)**1
a2 = 1/(1+0.04991*0.25)**2
a3 = 1/(1+0.05030*0.25)**3
a4 = 1/(1+0.05126*0.25)**4
a5 = 1/(1+0.05166*0.25)**5
a6 = 1/(1+0.05207*0.25)**6

In [6]:
P=[a1,a2,a3,a4,a5,a6]

In [7]:
hl1.fit(P,sigma=0.005,delta=0.25,compounding=1)

/Users/Ben/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [8]:
hl1.summary()

Fitted Interest Rates Tree:
         0         1         2         3         4         5
0  0.04969  0.052632  0.056085  0.061651  0.063276  0.066644
1  0.00000  0.047632  0.051085  0.056651  0.058276  0.061644
2  0.00000  0.000000  0.046085  0.051651  0.053276  0.056644
3  0.00000  0.000000  0.000000  0.046651  0.048276  0.051644
4  0.00000  0.000000  0.000000  0.000000  0.043276  0.046644
5  0.00000  0.000000  0.000000  0.000000  0.000000  0.041644
Fitted Prices Tree:
          0         1         2         3         4         5    6
0  0.925334  0.933939  0.943894  0.955360  0.968891  0.983612  1.0
1  0.000000  0.939718  0.948561  0.958898  0.971279  0.984823  1.0
2  0.000000  0.000000  0.953256  0.962452  0.973677  0.986037  1.0
3  0.000000  0.000000  0.000000  0.966024  0.976083  0.987254  1.0
4  0.000000  0.000000  0.000000  0.000000  0.978499  0.988473  1.0
5  0.000000  0.000000  0.000000  0.000000  0.000000  0.989696  1.0
6  0.000000  0.000000  0.000000  0.000000  0.000000  0.0

In [9]:
import numpy as np
CF=np.ones(len(P))
CF

array([1., 1., 1., 1., 1., 1.])

In [10]:
hl1.pricing(CF,type='fixed')[0]

5.739966705018595

In [11]:
hl1.pricing(CF,type='fixed')[1]

,0,1,2,3,4,5,6
0,5.739967,4.802444,3.859718,2.910264,1.953318,0.983612,0.0
1,0.000000,4.820099,3.871550,2.917408,1.956920,0.984823,0.0
2,0.000000,0.000000,3.883441,2.924581,1.960533,0.986037,0.0
3,0.000000,0.000000,0.000000,2.931784,1.964158,0.987254,0.0
4,0.000000,0.000000,0.000000,0.000000,1.967795,0.988473,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.989696,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [12]:
np.sum(P)

5.739966705018595

In [13]:
hl2=HoLee()

In [14]:
pzcb=[99.1338,97.8925,96.1462,94.1011,91.7136,89.2258,86.8142,84.5016,82.1848,79.7718,77.4339]
pzcb=[item/100 for item in pzcb]
pzcb

[0.9913379999999999,
 0.9789249999999999,
 0.9614619999999999,
 0.941011,
 0.917136,
 0.8922580000000001,
 0.868142,
 0.845016,
 0.8218479999999999,
 0.797718,
 0.7743389999999999]

In [15]:
help(hl2.fit)

Help on method fit in module __main__:

fit(P_zcb, sigma, delta, compounding=0) method of __main__.HoLee instance



In [16]:
hl2.fit(pzcb,0.0173,0.5)

/Users/Ben/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [17]:
hl2.summary()

Fitted Interest Rates Tree:
          0         1         2         3         4         5         6   \
0   0.017399  0.037471  0.060616  0.080036  0.100928  0.117101  0.129544   
1   0.000000  0.013005  0.036150  0.055570  0.076463  0.092635  0.105078   
2   0.000000  0.000000  0.011684  0.031104  0.051997  0.068169  0.080612   
3   0.000000  0.000000  0.000000  0.006638  0.027531  0.043703  0.056147   
4   0.000000  0.000000  0.000000  0.000000  0.003065  0.019237  0.031681   
5   0.000000  0.000000  0.000000  0.000000  0.000000 -0.005229  0.007215   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000 -0.017251   
7   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

          7         8         9         10  
0   0.141463  

In [18]:
def mycf(j,r):
    # only pays depends on r_10
    # max(11*100*r,94)
    if j==10:
        return max(11*100*r,94)
    else:
        return 0
mycf(10,0.18856)

207.416

In [19]:
hl2.pricing(mycf,type='conditional',lag=0)[0]

80.06443821654119

In [20]:
hl2.pricing(mycf,type='conditional',lag=0)[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,80.064438,79.046663,80.401412,84.486044,91.462084,101.703837,115.173691,131.660797,151.210368,174.883843,204.114951,0.0
1,0.000000,82.481370,80.681823,81.264958,84.409048,90.688345,100.499139,114.100482,131.410814,152.046714,177.202467,0.0
2,0.000000,0.000000,85.357122,83.041824,82.700089,84.708841,89.476047,97.740418,110.537726,128.620930,150.289983,0.0
3,0.000000,0.000000,0.000000,88.672629,85.986694,85.047848,85.815689,88.571895,94.199596,104.595459,123.377499,0.0
4,0.000000,0.000000,0.000000,0.000000,91.948174,89.309185,88.037767,87.946009,89.076599,91.733733,96.465015,0.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,94.869185,92.306949,90.940828,90.691886,91.660960,94.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.936025,94.340247,92.938110,92.789130,94.000000,0.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.866738,95.239968,93.931186,94.000000,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.598837,95.087298,94.000000,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,96.257640,94.000000,0.0
